In [ ]:
from rdflib import Graph, Dataset, URIRef
import re

In [ ]:
# the URI for our named graph that will contain our data
anno_named_graph_uri = URIRef('http://example.org/ns/users/robcast/genesis-example')

In [ ]:
# read the JSON-LD file with annotations from Recogito into a Graph
anno_graph = Graph()
anno_graph.parse('recogito-genesis-annotations.jsonld', 
                format='json-ld',
                publicId=anno_named_graph_uri)

In [ ]:
# write the Graph with annotations to a n-triples file
anno_graph.serialize('recogito-genesis-annotations.nt', format='nt', encoding='utf-8')

In [ ]:
# read the n-triples file and replace all blank nodes (starting with _:) with real URIs
# and write that to a new file
with open('recogito-genesis-annotations.nt', 'r') as infile, open('recogito-genesis-annotations-nb.nt', 'w') as outfile:
    for line in infile:
        line = re.sub(r'_:([^ ]+)', r'<http://example.org/ns/users/robcast/genesis-example-2/bnode_\1>', line)
        print(line, file=outfile)

In [ ]:
# open a connection to the Blazegraph triple store
blazegraph_uri = 'http://localhost:8080/blazegraph/namespace/kb/sparql'
store = Dataset('SPARQLUpdateStore')
store.open((blazegraph_uri, blazegraph_uri))

In [ ]:
# find our named graph if it exists or create a new one
anno_named_graph = None
for g in store.graphs():
    if g.identifier == anno_named_graph_uri:
        anno_named_graph = g

if anno_named_graph is None:
    anno_named_graph = store.graph(URIRef(anno_named_graph_uri))

In [ ]:
# parse our modified n-triples file into our named graph (takes a while)
anno_named_graph.parse('recogito-genesis-annotations-nb.nt')

In [ ]:
store.close()